# 8. 영화 추천시스템

먼저 프로젝트를 진행하기 전에 몇 가지 작업이 필요하다

1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip

## 8.1 데이터 준비와 전처리

In [1]:
# Movielens 데이터는 rading.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리 되어 있다
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 사용하는 colum만 남겨둔다
scol = ['user_id', 'movie_id', 'rating']
ratings = ratings[scol]
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
# 3점 이상의 평점의 영화만 남기도록 하자
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 counts로 바꿔준다
ratings.rename(columns={'rating':'counts'}, inplace=True)
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어온다
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 아까의 데이터와 합쳐준다
df = movies.set_index("movie_id")
ratings = ratings.join(df, on = "movie_id")

ratings

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama


## 8.2 데이터 분석

In [7]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [8]:
# rating에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# 가장 인기 있는 영화 30개 출력(인기순)
movie_count = ratings.groupby('title')['title'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 8.3 내가 선호하는 영화를 5가지 골라서 rating에 추가해 보자

In [10]:
idx_to_movie = {i:v for i,v in zip(movies['movie_id'], movies['title'])}
movie_to_idx = {v:i for i,v in zip(movies['movie_id'], movies['title'])}

In [11]:
# my favorite 리스트에 5개 골라서 집어넣어준다
my_favorite = [
'Fargo (1996)',                                       
'Braveheart (1995)',                                       
'Men in Black (1997)',
'L.A. Confidential (1997)',  
'Princess Bride, The (1987)'
]
my_idx = []
for i in my_favorite:
    my_idx.append(movie_to_idx[i])
my_idx
# MYJ라는 user_id가 위의 영화를 5회씩 봤다고 가정한다
my_playlist = pd.DataFrame({'user_id' : ['MYJ'] * 5, 'movie_id' : my_idx, 'title' : my_favorite, 'counts' : [5] * 5})

# user_id에 MYJ라는 데이터가 없다면 위에 임의로 만든 my_favorite 데이터를 추가해 준다
if not ratings.isin({'user_id' : ['MYJ']})['user_id'].any():
    ratings = ratings.append(my_playlist)
ratings.tail(10) 

,user_id,movie_id,counts,title,genre
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,MYJ,608,5,Fargo (1996),NaN
1,MYJ,110,5,Braveheart (1995),NaN
2,MYJ,1580,5,Men in Black (1997),NaN
3,MYJ,1617,5,L.A. Confidential (1997),NaN
4,MYJ,1197,5,"Princess Bride, The (1987)",NaN


In [12]:
# 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()
# 유저, 영화 indexing 하는 코드.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
movie_to_idx = {v : k for k, v in enumerate(movie_unique)}

In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구한다
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거한다 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 준다
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 준다
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')
ratings.tail(10)

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,counts,title,genre
1000203,6038,1030,3,Platoon (1986),Drama|War
1000205,6038,986,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6038,311,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6038,142,4,Sophie's Choice (1982),Drama
1000208,6038,26,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,6039,51,5,Fargo (1996),NaN
1,6039,87,5,Braveheart (1995),NaN
2,6039,175,5,Men in Black (1997),NaN
3,6039,224,5,L.A. Confidential (1997),NaN
4,6039,5,5,"Princess Bride, The (1987)",NaN


## 8.4  CSR matrix를 직접  만들어 보자

In [14]:
# CSR Matrix로 변환
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
print(num_user)
num_movie = ratings['movie_id'].nunique()
print(num_movie)

6040
3628


In [15]:
csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 8.5 als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 보자

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분이다
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=110, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해준다)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

## 8.6 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보자

In [20]:
# 모델이 1)Braveheart (1995)의 벡터를 어떻게 만들고 있는지와 
#       2)두 벡터를 곱하면 어떤 값이 나오는지 살펴보자

MYJ, brave = user_to_idx['MYJ'], movie_to_idx['Braveheart (1995)']
MYJ_vector, brave_vector = als_model.user_factors[MYJ], als_model.item_factors[brave]

In [21]:
MYJ_vector

array([ 0.62696606, -0.7111476 ,  0.11759786, -0.3461903 , -0.12474988,
       -0.30202746, -0.13105756,  0.82996076,  0.9088299 ,  0.30916315,
        0.21823844, -0.76107496,  0.22229025,  0.6088932 , -0.6077447 ,
       -0.47037885,  0.9769014 ,  0.47601417,  0.4799159 , -0.03537771,
       -0.24263942,  0.6294912 , -0.16237576, -0.06041238, -0.21169396,
        0.72675484, -0.33364615,  0.3658256 ,  0.50860494, -0.31313756,
        0.90220064,  0.75408804, -0.05007785, -0.15157725,  0.04024984,
        0.21861906, -0.4868819 ,  0.18586525,  0.3941096 , -0.3974963 ,
       -0.02130215,  0.20667508, -0.1424971 , -0.20060809,  0.09980612,
        0.4418907 ,  0.10082795, -1.031166  , -0.16052647, -0.4036669 ,
        0.08507218,  0.22622278,  0.3994001 , -0.83618176, -0.34489888,
        0.04687817, -0.40444148,  0.3973516 , -0.03865789, -0.8768555 ,
       -0.56369567, -0.21324643, -0.15911403, -0.32044527, -0.52857786,
       -0.60910356, -0.33530086,  0.09031103, -0.00984965,  0.04

In [22]:
brave_vector

array([ 0.05576075, -0.01981337,  0.00480203, -0.02298773,  0.00618316,
        0.02705522, -0.0056464 ,  0.04718376, -0.00214878,  0.01986295,
        0.0033997 , -0.007589  ,  0.00448925,  0.02155499, -0.03049257,
        0.00499038,  0.04990353,  0.03098584,  0.01127887,  0.00395251,
       -0.01847075,  0.0018512 , -0.01061727,  0.01103352, -0.02972314,
        0.00996851,  0.0096025 ,  0.01812674,  0.03151422, -0.00504972,
        0.04485822,  0.03109889,  0.00396623, -0.00650726,  0.01283871,
        0.0243304 ,  0.00665589,  0.03821611,  0.02268446, -0.02316785,
        0.00706408,  0.00687487,  0.03201428, -0.00127235,  0.03657791,
        0.01430178, -0.00524503, -0.01463719, -0.00340791,  0.00891433,
        0.05858524,  0.04025685,  0.00274612,  0.00448158, -0.03107355,
       -0.02570123, -0.02185572, -0.01165329,  0.01115551, -0.04792999,
       -0.01766757, -0.00649609,  0.01247535,  0.02484871, -0.01160716,
        0.01775107, -0.02179603,  0.03303676, -0.03338204,  0.01

In [23]:
# MYJ와 Braveheart (1995)를 내적하는 코드
np.dot(MYJ_vector, brave_vector)

0.6241399

In [24]:
# 모델에서는 나의 선호도를 0.57 정도로 판단한 것을 볼 수 있다.

## 8.7 내가 좋아하는 영화와 비슷한 영화를 추천받아 보자

In [25]:
# AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾는다.
# 처음으로는 Pulp Fiction에 대해서 찾아보겠다

favorite_movie = 'Pulp Fiction (1994)'

movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(222, 0.9999999),
 (269, 0.8045528),
 (51, 0.77478623),
 (233, 0.72399724),
 (157, 0.6756318),
 (121, 0.66731465),
 (289, 0.64701736),
 (224, 0.4863881),
 (248, 0.4862717),
 (220, 0.48234528),
 (278, 0.42469975),
 (23, 0.40964293),
 (59, 0.36584976),
 (483, 0.36187065),
 (806, 0.3326712)]

In [26]:
# 아티스트의 id, 유사도를 Tuple로 반환하고 있다. 그렇기 때문에 아티스트의 id를 다시 아티스트의 이름으로 매핑 시켜 준다.
# artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성

idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Pulp Fiction (1994)',
 'GoodFellas (1990)',
 'Fargo (1996)',
 'Usual Suspects, The (1995)',
 'Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Reservoir Dogs (1992)',
 'L.A. Confidential (1997)',
 'Good Will Hunting (1997)',
 'Seven (Se7en) (1995)',
 'Jackie Brown (1997)',
 "Schindler's List (1993)",
 'Few Good Men, A (1992)',
 'Trainspotting (1996)',
 'Striptease (1996)']

In [27]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠다
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [28]:
get_similar_movie('Silence of the Lambs, The (1991)')

['Silence of the Lambs, The (1991)',
 'Shawshank Redemption, The (1994)',
 'Fargo (1996)',
 'Pulp Fiction (1994)',
 'Usual Suspects, The (1995)',
 'GoodFellas (1990)',
 "Schindler's List (1993)",
 'Sixth Sense, The (1999)',
 'Good Will Hunting (1997)',
 'Sling Blade (1996)']

## 8.8 내가 가장 좋아할 만한 영화들을 추천받아 보자

In [29]:
# 유저에게 영화 추천하기
# AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 좋아할 만한 아티스트를 추천받는다.
# filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument다.
user = user_to_idx['MYJ']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=10, filter_already_liked_items=True)
movie_recommended

[(107, 0.5383196),
 (92, 0.4305806),
 (121, 0.41334558),
 (222, 0.37125534),
 (233, 0.36648944),
 (80, 0.3439396),
 (157, 0.34241444),
 (99, 0.33135116),
 (124, 0.3309847),
 (48, 0.29729056)]

In [30]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Silence of the Lambs, The (1991)',
 'Pulp Fiction (1994)',
 'Usual Suspects, The (1995)',
 'Stand by Me (1986)',
 'Shawshank Redemption, The (1994)',
 'American Beauty (1999)',
 'Matrix, The (1999)',
 'Saving Private Ryan (1998)']

In [31]:
# 모델은 여러개의 영화를 추천해줬는데 왜 이것들을 추천해줬는지 알아보자
park = movie_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=park)

In [32]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.35056134888893575),
 ('Braveheart (1995)', 0.15912352253182535),
 ('Fargo (1996)', 0.01064930795932474),
 ('L.A. Confidential (1997)', 0.009617515814918862),
 ('Princess Bride, The (1987)', 0.0002975621180058372)]

In [33]:
# Men in Black (1997)과 Braveheart (1995)가 가장 크게 기여한 것을 볼 수 있다.

## 회고